In [34]:
import os
import simpleaudio as sa
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langgraph.prebuilt import create_react_agent
import whisper
from TTS.api import TTS
import sounddevice as sd
from scipy.io.wavfile import write

In [35]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [36]:
model = ChatGroq(model="llama3-70b-8192", temperature=0.2)
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a helpful assistant. If you don't know something, use the DuckDuckGo tool to search the web. "
     "When using the tool, always pass a clear and non-empty 'query' argument string."),
    MessagesPlaceholder(variable_name="messages")
])
memory = MemorySaver()

In [37]:
duck_api = DuckDuckGoSearchAPIWrapper(max_results=1,safesearch="moderate")
duck = DuckDuckGoSearchRun(api_wrapper=duck_api)
tools = [duck,YahooFinanceNewsTool()]
agent_executor = create_react_agent(model,prompt=prompt, tools=tools, checkpointer=memory)

In [38]:
config = {
    "configurable": {
        "thread_id": "abc124"
        }
    }

In [39]:
def record_audio(file_path="input.wav", duration=6, fs=16000):
    print("🎙️ Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    write(file_path, fs, audio)
    print(f"✅ Saved recording to {file_path}")
    print("📁 File exists:", os.path.exists(file_path))


In [40]:
def transcribe(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    print("📝 Transcription:", result["text"])
    return result["text"]

In [41]:
def ask_agent(prompt):
    result = agent_executor.invoke({"messages": prompt}, config=config)
    return result["messages"][-1].content

In [42]:
def speak(text, file="reply.wav"):
    tts = TTS("tts_models/en/ljspeech/vits")  
    tts.tts_to_file(text=text, file_path=file)
    print("🔊 TTS complete")

In [43]:
def play_audio(file="reply.wav"):
    wave_obj = sa.WaveObject.from_wave_file(file)
    play_obj = wave_obj.play()
    play_obj.wait_done()

In [ ]:
if __name__ == "__main__":
    record_audio()
    query = transcribe_audio("input.wav")
    answer = ask_agent(query)
    print("🤖 Bot:", answer)
    speak(answer)
    play_audio()

🎙️ Recording...
✅ Saved recording to input.wav
📁 File exists: True
🧠 Transcribing with Whisper...
📝 Transcribed Text:  Hello Myself Abhinav Sunil. I am a Computer Diwali Student from Diwali Professional University.
🤖 Bot: It seems like there might be some confusion. Diwali Professional University doesn't seem to be a real university. Lovely Professional University is a well-known university in India, and it seems like the tool yielded a result related to that university instead.
 > tts_models/en/ljspeech/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | 

g:\TTS\lib\site-packages\TTS\utils\io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Text splitted to sentences.
['It seems like there might be some confusion.', "Diwali Professional University doesn't seem to be a real university.", 'Lovely Professional University is a well-known university in India, and it seems like the tool yielded a result related to that university instead.']
 > Processing time: 7.1545140743255615
 > Real-time factor: 0.38486337127444137
🔊 TTS complete


: 